In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/jerimee/north-carolina-voter-file")

In [ ]:
import pandas as pd

nc = "north-carolina-voter-file\\ncvoter_Statewide\\ncvoter_Statewide.txt"
nc = pd.read_csv(nc, delimiter = '\t')

In [ ]:
for col in nc.columns:
    print(col)

In [ ]:
cols = ['cong_dist_abbrv',
'party_cd',
'status_cd',
'precinct_abbrv']

nc_voters = nc[cols]
nc_voters.head()

In [ ]:
nc_voters['party_cd'].unique()

#NC Parties = Undeclared, Constitution, Democratic, Green, Libertarian and Republican
#To match our other data, I'm changing to OTH, CONST, DEM, GRN, LBT, and REP

nc_voters['party_cd'] = nc_voters['party_cd'].replace(['UNA','DEM','REP','LIB','GRE','CST'],
                                                      ['OTH','DEM','REP','LBT','GRN','CONST'])

In [ ]:
#keeping only active voters
nc_voters = nc_voters[nc_voters['status_cd']=='A']
nc_voters = nc_voters.drop(columns='status_cd', axis=1)

In [ ]:
#Aggregate by county/precinct/party
nc_totals = nc_voters.groupby(['cong_dist_abbrv', 'precinct_abbrv','party_cd']).size().reset_index(name='counts')

In [ ]:
nc_totals['STATE']='NC'
nc_totals.head()

In [ ]:
nc_totals = nc_totals.rename(columns={'cong_dist_abbrv':'DISTRICT', 
                                      'precinct_abbrv':'PRECINCT','party_cd':'PARTY','counts':'VOTERS'})
nc_totals = nc_totals[['STATE','PRECINCT','DISTRICT','PARTY','VOTERS']]

In [ ]:
# Inspo from: https://www.youtube.com/watch?v=iOurqfCJwy8
# but I changed it so I didn't have to explicitly call the columns in the print statement

l = []
for col in nc_totals.columns:
    x=(col)
    l.append(x)

with open('ncinsert.sql','w'):
    for index, row in nc_totals.iterrows():
        print('INSERT INTO precinct'+'('+str(l).strip('[]')+') '+'VALUES('+
              '"'+str(row[str(l[0])])+'", '+
              '"'+str(row[str(l[1])])+'", '+
              '"'+str(row[str(l[2])])+'", '+
              '"'+str(row[str(l[3])])+'", '+
              str(row[str(l[4])])+
              ');',file=open('ncinsert.sql','a'))

#Adding the begin transactoin and commit lines to the original file
with open('ncinsert.sql', 'r') as original: data = original.read()
with open('ncinsert.sql', 'w') as modified: modified.write("BEGIN TRANSACTION;\n" + data + "COMMIT;")

In [ ]:
ri = "https://raw.githubusercontent.com/anita-uva/gerrymander/main/state-data/RI/ri.txt"
ri = pd.read_csv(ri, sep="      ", engine='python')

In [ ]:
for col in ri.columns:
    print(col)

In [ ]:
ri.columns = ri.columns.str.replace(' ', '')

In [ ]:
for col in ri.columns:
    print(col)

In [ ]:
cols = ['City/Town','Precinct','Party','Reg.Voters']
ri=ri[cols]

ri.head()

In [ ]:
ri['STATE']='RI'
ri_totals = ri.rename(columns={'City/Town':'DISTRICT', 
                                      'Precinct':'PRECINCT','Party':'PARTY','Reg.Voters':'VOTERS'})
ri_totals = ri_totals[['STATE','PRECINCT','DISTRICT','PARTY','VOTERS']]
#Note: RI has unique precinct numbers, so District is not needed.  
#Leaving City/Town in its place for geographical purposes

In [ ]:
ri_totals['PARTY']=ri_totals['PARTY'].str.strip()
ri_totals['PARTY'] = ri_totals['PARTY'].replace(['Republican','Democrat','Unaffiliated'],
                                                      ['DEM','REP','OTH'])

In [ ]:
import numpy as np
ri_totals['VOTERS']=ri_totals['VOTERS'].str.strip()
ri_totals['VOTERS'] = ri_totals['VOTERS'].replace(['(Blank)'],[np.nan])
ri_totals['VOTERS']=pd.to_numeric(ri_totals['VOTERS'])

In [ ]:
ri_totals = ri_totals.dropna()
# Inspo from: https://www.youtube.com/watch?v=iOurqfCJwy8
# but I changed it so I didn't have to explicitly call the columns in the print statement

l = []
for col in ri_totals.columns:
    x=(col)
    l.append(x)

with open('riinsert.sql','w'):
    for index, row in ri_totals.iterrows():
        print('INSERT INTO precinct'+'('+str(l).strip('[]')+') '+'VALUES('+
              '"'+str(row[str(l[0])])+'", '+
              '"'+str(row[str(l[1])])+'", '+
              '"'+str(row[str(l[2])])+'", '+
              '"'+str(row[str(l[3])])+'", '+
              str(row[str(l[4])])+
              ');',file=open('riinsert.sql','a'))

#Adding the begin transactoin and commit lines to the original file
with open('riinsert.sql', 'r') as original: data = original.read()
with open('riinsert.sql', 'w') as modified: modified.write("BEGIN TRANSACTION;\n" + data + "COMMIT;")